In [432]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [433]:
df = pd.read_csv('SP500 IFPS Price History.txt')
df.head()

,timestamp,IFPID,Outcome,Share Price,Trade Qty
0,2022-05-05 04:14:44,SP4K5K,up,44,100
1,2022-05-05 04:14:44,SP4K5K,dn,56,100
2,2022-05-05 15:53:59,SP4K5K,dn,90,5
3,2022-05-05 15:53:59,SP4K5K,up,10,5
4,2022-05-05 17:17:59,SP4K5K,dn,85,100


In [434]:
df['Date'] = pd.to_datetime(df['timestamp']).dt.date
df.head()

,timestamp,IFPID,Outcome,Share Price,Trade Qty,Date
0,2022-05-05 04:14:44,SP4K5K,up,44,100,2022-05-05
1,2022-05-05 04:14:44,SP4K5K,dn,56,100,2022-05-05
2,2022-05-05 15:53:59,SP4K5K,dn,90,5,2022-05-05
3,2022-05-05 15:53:59,SP4K5K,up,10,5,2022-05-05
4,2022-05-05 17:17:59,SP4K5K,dn,85,100,2022-05-05


In [435]:
df['Date'] = df["Date"].astype('datetime64[ns]')

In [436]:
df.tail()

,timestamp,IFPID,Outcome,Share Price,Trade Qty,Date
518,2022-11-10 16:35:46,SP3K4K,up,90,50,2022-11-10
519,2022-11-11 16:04:37,SP3K4K,up,91,20,2022-11-11
520,2022-11-11 16:04:37,SP3K4K,up,90,10,2022-11-11
521,2022-11-11 16:35:02,SP3K4K,up,99,150,2022-11-11
522,2022-11-11 16:35:02,SP3K4K,up,94,20,2022-11-11


In [437]:
df.shape

(523, 6)

In [438]:
df.dtypes

timestamp              object
IFPID                  object
Outcome                object
Share Price             int64
Trade Qty               int64
Date           datetime64[ns]
dtype: object

In [439]:
df.nunique()

timestamp      274
IFPID            3
Outcome          2
Share Price     92
Trade Qty       80
Date           111
dtype: int64

Aggregated sum by outcome

In [440]:
df['agg_sum_by_outcome']  = df.groupby(['Date','Outcome'])['Trade Qty'].transform(np.sum)

In [441]:
df.head()

,timestamp,IFPID,Outcome,Share Price,Trade Qty,Date,agg_sum_by_outcome
0,2022-05-05 04:14:44,SP4K5K,up,44,100,2022-05-05,105
1,2022-05-05 04:14:44,SP4K5K,dn,56,100,2022-05-05,205
2,2022-05-05 15:53:59,SP4K5K,dn,90,5,2022-05-05,205
3,2022-05-05 15:53:59,SP4K5K,up,10,5,2022-05-05,105
4,2022-05-05 17:17:59,SP4K5K,dn,85,100,2022-05-05,205


Assigning labels to the developed strategy by first aggregating the means of the outcome variable, then compare their means when the mean of up is higher than that of down we predict a buy while when is lower we predict a sell

In [442]:
df_piv = df.pivot_table(index="Date", columns="Outcome", values="Trade Qty", aggfunc='median', fill_value=0)
df_piv['Predicted'] = np.select([df_piv['dn']<df_piv['up'],df_piv['dn']>df_piv['up']], ['sell','buy'])
df_piv

Outcome,dn,up,Predicted
Date,,,
2022-05-05,100.0,52.5,buy
2022-05-06,5.0,5.0,0
2022-05-07,10.0,10.0,0
2022-05-08,7.0,5.0,buy
2022-05-09,125.0,345.0,sell
...,...,...,...
2022-11-03,50.0,50.0,0
2022-11-04,134.5,83.0,buy
2022-11-07,130.0,90.0,buy


In [443]:
df_piv.columns

Index(['dn', 'up', 'Predicted'], dtype='object', name='Outcome')

In [444]:
df_piv.reset_index()

Outcome,Date,dn,up,Predicted
0,2022-05-05,100.0,52.5,buy
1,2022-05-06,5.0,5.0,0
2,2022-05-07,10.0,10.0,0
3,2022-05-08,7.0,5.0,buy
4,2022-05-09,125.0,345.0,sell
...,...,...,...,...
106,2022-11-03,50.0,50.0,0
107,2022-11-04,134.5,83.0,buy
108,2022-11-07,130.0,90.0,buy
109,2022-11-10,235.5,43.5,buy


# ACTUAL DATA FROM S P 500

In [445]:
df2 = pd.read_csv('SP5002.csv')

In [446]:
df2.head()

,Date,Open,High,Low,Close
0,12/30/2022,"3,829.06","3,839.85","3,800.34","3,839.50"
1,12/29/2022,"3,805.45","3,858.19","3,805.45","3,849.28"
2,12/28/2022,"3,829.56","3,848.32","3,780.78","3,783.22"
3,12/27/2022,"3,843.34","3,846.65","3,813.22","3,829.25"
4,12/23/2022,"3,813.11","3,845.80","3,797.01","3,844.82"


In [447]:
df2.tail()

,Date,Open,High,Low,Close
248,01/05/2022,"4,787.99","4,797.70","4,699.44","4,700.58"
249,01/04/2022,"4,804.51","4,818.62","4,774.27","4,793.54"
250,01/03/2022,"4,778.14","4,796.64","4,758.17","4,796.56"
251,12/31/2021,"4,775.21","4,786.83","4,765.75","4,766.18"
252,12/30/2021,"4,794.23","4,808.93","4,775.33","4,778.73"


In [448]:
df2.dtypes

Date     object
Open     object
High     object
Low      object
Close    object
dtype: object

In [449]:
df2.nunique()

Date     253
Open     253
High     253
Low      253
Close    253
dtype: int64

# CONVERTING THE OBJECT DATA TYPE TO FLOAT AND DATETIME

In [450]:
df2["Open"] = [float(str(i).replace(",", "")) for i in df2["Open"]]

In [451]:
df2["High"] = [float(str(i).replace(",", "")) for i in df2["High"]]

In [452]:
df2["Low"] = [float(str(i).replace(",", "")) for i in df2["Low"]]

In [453]:
df2["Close"] = [float(str(i).replace(",", "")) for i in df2["Close"]]

In [454]:
df2['Date'] = df2["Date"].astype('datetime64[ns]')

CALCULATING THE AVERAGE PRICE FOR EACH DAY

In [455]:
df2['Average'] = (df2['Open'] + df2['High'] + df2['Low'] + df2['Close'])/4

In [456]:
df2.dtypes

Date       datetime64[ns]
Open              float64
High              float64
Low               float64
Close             float64
Average           float64
dtype: object

In [457]:
df2.head()

,Date,Open,High,Low,Close,Average
0,2022-12-30,3829.06,3839.85,3800.34,3839.50,3827.1875
1,2022-12-29,3805.45,3858.19,3805.45,3849.28,3829.5925
2,2022-12-28,3829.56,3848.32,3780.78,3783.22,3810.4700
3,2022-12-27,3843.34,3846.65,3813.22,3829.25,3833.1150
4,2022-12-23,3813.11,3845.80,3797.01,3844.82,3825.1850


SORTING THE DATE COLUMN BY ASCENDING ORDER

In [458]:
df2.sort_values(by='Date', inplace = True)

In [459]:
df2.head()

,Date,Open,High,Low,Close,Average
252,2021-12-30,4794.23,4808.93,4775.33,4778.73,4789.3050
251,2021-12-31,4775.21,4786.83,4765.75,4766.18,4773.4925
250,2022-01-03,4778.14,4796.64,4758.17,4796.56,4782.3775
249,2022-01-04,4804.51,4818.62,4774.27,4793.54,4797.7350
248,2022-01-05,4787.99,4797.70,4699.44,4700.58,4746.4275


ASSIGNING LABELS TO THE ACTUAL MARKET SCENERIO BY COMPARING THE CLOSING PRICE WITH THE OPENING PRICE, IF THE CLOSING PRICE IS HIGHER THAN THE OPENING THEN THAT INDICATES THAT THE MARKET WAS A BUY WHILE WHEN IT THE REVERSE THAT IS THE CLOSING PRICE IS LOWER THAN THE OPENING PRICE THEN IT A SELL

In [460]:
df2.loc[:,'Actual'] = 'buy'
df2.loc[df2['Close']<df2['Open'],'Actual'] = 'sell'
df2.head()

,Date,Open,High,Low,Close,Average,Actual
252,2021-12-30,4794.23,4808.93,4775.33,4778.73,4789.3050,sell
251,2021-12-31,4775.21,4786.83,4765.75,4766.18,4773.4925,sell
250,2022-01-03,4778.14,4796.64,4758.17,4796.56,4782.3775,buy
249,2022-01-04,4804.51,4818.62,4774.27,4793.54,4797.7350,sell
248,2022-01-05,4787.99,4797.70,4699.44,4700.58,4746.4275,sell


In [461]:
#df2.drop([111,110,109], axis=0, inplace=True)

In [462]:
df2.head()

,Date,Open,High,Low,Close,Average,Actual
252,2021-12-30,4794.23,4808.93,4775.33,4778.73,4789.3050,sell
251,2021-12-31,4775.21,4786.83,4765.75,4766.18,4773.4925,sell
250,2022-01-03,4778.14,4796.64,4758.17,4796.56,4782.3775,buy
249,2022-01-04,4804.51,4818.62,4774.27,4793.54,4797.7350,sell
248,2022-01-05,4787.99,4797.70,4699.44,4700.58,4746.4275,sell


# COMPARING THE ACTUAL MARKET SCENERIO WITH OUR DEVELOPED STRATEGY

HERE WE WOULD MERGE THE PREDICTED STRATEGY BY OUR ALGORITHM WITH THE ACTUAL MARKET SCENERIO ABOVE

In [463]:
df_strategy = df_piv.reset_index('Date')

In [464]:
df_strategy.head()

Outcome,Date,dn,up,Predicted
0,2022-05-05,100.0,52.5,buy
1,2022-05-06,5.0,5.0,0
2,2022-05-07,10.0,10.0,0
3,2022-05-08,7.0,5.0,buy
4,2022-05-09,125.0,345.0,sell


We would drop all other columns leaving just the date and predicted outcome

In [465]:
df_strategy.drop(columns = ['dn','up'], inplace=True)

In [466]:
df_strategy.shape

(111, 2)

In [467]:
df_strategy.head()

Outcome,Date,Predicted
0,2022-05-05,buy
1,2022-05-06,0
2,2022-05-07,0
3,2022-05-08,buy
4,2022-05-09,sell


In [468]:
df_Actual = df2

In [469]:
df_Actual.head()

,Date,Open,High,Low,Close,Average,Actual
252,2021-12-30,4794.23,4808.93,4775.33,4778.73,4789.3050,sell
251,2021-12-31,4775.21,4786.83,4765.75,4766.18,4773.4925,sell
250,2022-01-03,4778.14,4796.64,4758.17,4796.56,4782.3775,buy
249,2022-01-04,4804.51,4818.62,4774.27,4793.54,4797.7350,sell
248,2022-01-05,4787.99,4797.70,4699.44,4700.58,4746.4275,sell


We would drop all other columns leaving just the date actual outcome

In [470]:
df_Actual.drop(columns = ['Open','High','Low','Close','Average'], inplace=True)

In [471]:
df_Actual.head()

,Date,Actual
252,2021-12-30,sell
251,2021-12-31,sell
250,2022-01-03,buy
249,2022-01-04,sell
248,2022-01-05,sell


In [472]:
new_df = pd.merge(df_strategy,df_Actual, on='Date')

In [473]:
new_df.head()

,Date,Predicted,Actual
0,2022-05-05,buy,sell
1,2022-05-06,0,sell
2,2022-05-09,sell,sell
3,2022-05-10,0,sell
4,2022-05-11,buy,sell


In [474]:
new_df.shape

(85, 3)

In [475]:
new_df.isnull().sum()

Date         0
Predicted    0
Actual       0
dtype: int64

In [476]:
new_df= new_df[~(new_df["Predicted"] == "0")]

In [477]:
new_df['Outcome'] = np.select([new_df['Predicted']==new_df['Actual'],new_df['Predicted']!=new_df['Actual']], ['Yes','No'])

In [478]:
new_df.head()

,Date,Predicted,Actual,Outcome
0,2022-05-05,buy,sell,No
2,2022-05-09,sell,sell,Yes
4,2022-05-11,buy,sell,No
5,2022-05-12,buy,buy,Yes
6,2022-05-13,buy,buy,Yes


In [479]:
new_df['Outcome'].value_counts()

Yes    32
No     29
Name: Outcome, dtype: int64

BASED ON THIS WORK THE ACCURACY OF OUR ALGORITHM IS 63% WHICH IS CALCULATED BY DIVIDING THE NUMBER OF YES 39 BY THE TOTAL NUMBER OF OBSERVATIONS 83 THEN MULTIPLY BY 100

In [480]:
new_df2 = pd.merge(df_strategy,df_Actual, on='Date', how='outer')

In [481]:
new_df2.head()

,Date,Predicted,Actual
0,2022-05-05,buy,sell
1,2022-05-06,0,sell
2,2022-05-07,0,NaN
3,2022-05-08,buy,NaN
4,2022-05-09,sell,sell


In [482]:
new_df2.shape

(279, 3)

In [483]:
new_df2.isnull().sum()

Date           0
Predicted    168
Actual        26
dtype: int64

In [484]:
new_df3 = pd.merge(df_strategy,df_Actual, on='Date', how='left')

In [485]:
new_df3.shape

(111, 3)

In [486]:
new_df3.isnull().sum()

Date          0
Predicted     0
Actual       26
dtype: int64